In [1]:
import geopandas as gpd

In [3]:
import zipfile 
import os
zip_path = 'mrds-trim.zip' 
extract_to = 'data'
os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref: zip_ref.extractall(extract_to)

In [4]:
#find the names of the files I just unzipped
for root, dirs, files in os.walk(extract_to):
    for file in files:
        print(os.path.join(root, file))

data/mrds-trim.shp
data/mrds-trim.shx
data/mrds.met
data/mrds-trim.dbf
data/mrds-trim.prj
data/mrds-trim.lyr


In [5]:
minerals = gpd.read_file("data/mrds-trim.shp")

In [6]:
print(minerals.shape)

(304632, 6)


In [7]:
print(minerals.columns)

Index(['DEP_ID', 'SITE_NAME', 'DEV_STAT', 'URL', 'CODE_LIST', 'geometry'], dtype='object')


In [8]:
print(minerals.head())

     DEP_ID            SITE_NAME    DEV_STAT  \
0  10000001     Lookout Prospect  Occurrence   
1  10000002  Lucky Find Prospect  Occurrence   
2  10000003  Mccullough Prospect  Occurrence   
3  10000004      Lucky Jim Claim  Occurrence   
4  10000005   Matilda Occurrence  Occurrence   

                                                 URL CODE_LIST  \
0  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...  CU AU AG   
1  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...     CU AU   
2  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...  CU ZN AU   
3  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...  AU PB CU   
4  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...        AU   

                      geometry  
0  POINT (-132.14344 55.05612)  
1  POINT (-132.68514 55.52751)  
2  POINT (-132.99906 55.97751)  
3  POINT (-132.68653 55.52195)  
4  POINT (-132.05233 55.14556)  


In [14]:
#we want to find the mines that do contain lithium as a preliminary investigation of what other minerals are there
filtered_minerals = minerals[minerals['CODE_LIST'].str.contains('LI', na=False)]

In [15]:
print(filtered_minerals.shape)

(566, 6)


In [16]:
print(filtered_minerals.head())

        DEP_ID                SITE_NAME       DEV_STAT  \
4108  10004123             La Teresaida     Occurrence   
4156  10004198                   Ipizca     Occurrence   
4157  10004199  Salar Del Hombre Muerto     Occurrence   
4159  10004201         Reflejos Del Mar     Occurrence   
4165  10004207              Victor Hugo  Past Producer   

                                                    URL  \
4108  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...   
4156  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...   
4157  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...   
4159  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...   
4165  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...   

                   CODE_LIST                     geometry  
4108       LI QTZ FLD MIC NB  POINT (-65.83271 -32.66411)  
4156                   LI BE  POINT (-65.54107 -28.72578)  
4157            LI K B CS RB  POINT (-66.99944 -25.24749)  
4159                   LI BE   POINT (-65.4494 -28.44745

In [17]:
#now we want to look at the mines that do not contain lithium 
no_lithium_minerals = minerals[~minerals['CODE_LIST'].str.contains('LI', na=False)]

In [18]:
print(no_lithium_minerals.shape)

(304066, 6)


In [19]:
print(no_lithium_minerals.head())

     DEP_ID            SITE_NAME    DEV_STAT  \
0  10000001     Lookout Prospect  Occurrence   
1  10000002  Lucky Find Prospect  Occurrence   
2  10000003  Mccullough Prospect  Occurrence   
3  10000004      Lucky Jim Claim  Occurrence   
4  10000005   Matilda Occurrence  Occurrence   

                                                 URL CODE_LIST  \
0  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...  CU AU AG   
1  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...     CU AU   
2  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...  CU ZN AU   
3  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...  AU PB CU   
4  https://mrdata.usgs.gov/mrds/show-mrds.php?dep...        AU   

                      geometry  
0  POINT (-132.14344 55.05612)  
1  POINT (-132.68514 55.52751)  
2  POINT (-132.99906 55.97751)  
3  POINT (-132.68653 55.52195)  
4  POINT (-132.05233 55.14556)  


In [20]:
#we want to restrict to only north carolina as part of our first exploratory analysis
#we know from the North Carolina wikipedia page the longitude and latitude of the state bounds: https://en.wikipedia.org/wiki/North_Carolina

In [65]:

min_lat, max_lat = 33.8333, 36.5833
min_lon, max_lon = -84.3167, -75.4667


minerals['longitude'] = minerals.geometry.x
minerals['latitude'] = minerals.geometry.y


minerals_nc = minerals[
    (minerals['latitude'] >= min_lat) & (minerals['latitude'] <= max_lat) &
    (minerals['longitude'] >= min_lon) & (minerals['longitude'] <= max_lon)
]

In [66]:
#before we do calculate any distances with our data we should first convert to a north carolina specific epsg for greater spatial accuracy
minerals_nc_projected = minerals_nc.to_crs(epsg=32119)

#get rid of processing sites since minerals not explicitly mined there
#get rid of sites that just have SDG as their only output since that's sand and gravel rather than a mineral
minerals_nc_projected = minerals_nc_projected[
    ~minerals_nc_projected['DEV_STAT'].isin(['Unknown', 'Plant']) &
    ~(minerals_nc_projected['CODE_LIST'] == 'SDG')
]

In [67]:
#now within north carolina we want to pick one mine that contains lithium and then see distances to the other mines that do not have lithium
#first we find one mine that contains lithium
li_mines = minerals_nc_projected[minerals_nc_projected['CODE_LIST'].str.contains('LI', na=False)]
print(li_mines.iloc[0])

DEP_ID                                                10024735
SITE_NAME                                        Bessemer City
DEV_STAT                                              Prospect
URL          https://mrdata.usgs.gov/mrds/show-mrds.php?dep...
CODE_LIST                                        LI FLD MIC BE
geometry         POINT (411158.29533077776 177832.56568582306)
longitude                                            -81.18315
latitude                                              35.33343
Name: 23804, dtype: object


In [68]:
#filtering for just mines that do not contain lithium
non_li_mines = minerals_nc_projected[~minerals_nc_projected['CODE_LIST'].str.contains('LI', na=False)]

In [69]:
non_li_mines.shape

(6917, 8)

In [70]:
bessemer_city_geom = minerals_nc_projected.loc[23804, 'geometry']

In [71]:
print(bessemer_city_geom)

POINT (411158.29533077776 177832.56568582306)


In [72]:
non_li_mines = non_li_mines.copy()
non_li_mines['distance_to_bessemer'] = non_li_mines.geometry.distance(bessemer_city_geom)

In [73]:
#find the closest 5 non-lithium containing mines to those mines: 
print(non_li_mines[['DEP_ID', 'SITE_NAME', 'distance_to_bessemer', "CODE_LIST", "DEV_STAT"    ]].sort_values('distance_to_bessemer').head())

          DEP_ID                            SITE_NAME  distance_to_bessemer  \
77168   10079365          Unnamed Stone Quarry Fk-045           5656.124332   
289697  10297157                          Froneberger           6736.869397   
76774   10078901    Spencer Mountain Quartzite Quarry           7203.166211   
77009   10079138                    Unnamed Gold Mine           7338.096643   
76772   10078899  Spencer Mountain Kyanite Occurrence           7403.812414   

       CODE_LIST       DEV_STAT  
77168        STN       Producer  
289697        MN       Prospect  
76774        STN  Past Producer  
77009         AU  Past Producer  
76772    QTZ KYN     Occurrence  


In [74]:
#now find the closest 20:
print(
    non_li_mines[
        ['DEP_ID', 'SITE_NAME', 'distance_to_bessemer', 'CODE_LIST', 'DEV_STAT']
    ].sort_values('distance_to_bessemer').head(20)
)

          DEP_ID                            SITE_NAME  distance_to_bessemer  \
77168   10079365          Unnamed Stone Quarry Fk-045           5656.124332   
289697  10297157                          Froneberger           6736.869397   
76774   10078901    Spencer Mountain Quartzite Quarry           7203.166211   
77009   10079138                    Unnamed Gold Mine           7338.096643   
76772   10078899  Spencer Mountain Kyanite Occurrence           7403.812414   
274491  10281543                      Long Creek Mine           7592.381125   
87224   10089674                    Unnamed Gold Mine           7596.625900   
97512   10100209                    Unnamed Gold Mine           7651.639280   
123712  10127217                          Oliver Mine           7877.521503   
171070  10175745                         Costner Mine           7987.132254   
194561  10199779    Walter and Gerard Pesaur Property           8010.439015   
170742  10175417                         Ellison Min

In [75]:
#These are close mines which means that lithium mines are near mines that do not contain lithium, so we can proceed!